In [ ]:
!pip install langchain openai unstructured pdf2image tiktoken apify-client pinecone-client

In [63]:
from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain.utilities import ApifyWrapper
from langchain.vectorstores import Pinecone
import os
from langchain.chains import RetrievalQA
import pinecone
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI



In [4]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    """
    Split the given list of documents into chunks based on the specified chunk size and overlap.

    Args:
        documents (list): List of documents to be split.
        chunk_size (int): Size of each chunk (default: 1000).
        chunk_overlap (int): Overlap between consecutive chunks (default: 20).

    Returns:
        list: List of split documents.
    """
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs


In [7]:
import os
os.environ["OPENAI_API_KEY"] = "sk-XYNkKwsIETv"
os.environ["APIFY_API_TOKEN"] = "apify_api"

In [8]:
apify = ApifyWrapper()
embeddings = OpenAIEmbeddings(model="gpt-3.5-turbo")


pinecone.init(
    api_key="0b596385-53",
    environment="us-east-1-aws"
)
index_name = "new"


In [ ]:
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "https://writtenbykaushal.hashnode.dev/"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

In [ ]:
# Load the data from the loader
data = loader.load()


In [ ]:
docs = split_docs(data)

In [ ]:
#This like is used for creating embedding of our data. We have to use it only once at beginning.
#index = Pinecone.from_documents(docs, embeddings, index_name=index_name)


In [ ]:
# Extract the URLs
scraped_urls = [doc.metadata["source"] for doc in data]

print(scraped_urls)

['https://writtenbykaushal.hashnode.dev/', 'https://writtenbykaushal.hashnode.dev/rss.xml', 'https://writtenbykaushal.hashnode.dev/about-me', 'https://writtenbykaushal.hashnode.dev/newsletter', 'https://writtenbykaushal.hashnode.dev/from-fomo-to-lol-exploring-the-wonders-of-llms', 'https://writtenbykaushal.hashnode.dev/vector-databases-the-magic-potion-behind-chatgpt-and-jokes', 'https://writtenbykaushal.hashnode.dev/the-future-of-chatbots-building-a-state-of-the-art-stateless-chatbot-with-openais-gpt-and-streamlit', 'https://writtenbykaushal.hashnode.dev/breaking-down-the-misconceptions-the-truth-about-chat-gpts-understanding-of-language', 'https://writtenbykaushal.hashnode.dev/how-to-extract-features-from-the-text-for-sentiment-analysis-a-beginners-guide-to-making-sense-of-text-data-to-computer', 'https://writtenbykaushal.hashnode.dev/what-is-dirty-data-what-are-its-types-9a316aab0ef3', 'https://writtenbykaushal.hashnode.dev/from-error-messages-to-life-lessons-a-journey-in-computer-s

In [52]:
#Load saved embeddings
index = Pinecone.from_existing_index(index_name, embeddings) # This lines gets saved embeddings


In [64]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [70]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), index.as_retriever(), return_source_documents=True)

In [71]:
chat_history = []
query = "Who wrote these blogs?"
result = qa({"question": query, "chat_history": chat_history})

In [73]:
result['answer']

' The blogs were written by Kaushal.'

In [72]:
result['source_documents'][0]


Document(page_content='Tag\nProgramming Blogs\n#programming-blogs\nMore content\nRead more stories on Hashnode', metadata={'source': 'https://writtenbykaushal.hashnode.dev/tag/programming-blogs'})